In [ ]:
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

In [ ]:
with open("QBdata", 'rb') as picklefile: 
    statsDF = pkl.load(picklefile)

with open("fantasyDEFdata", 'rb') as picklefile: 
    defDF = pkl.load(picklefile)

In [ ]:
# reduce DF to QBs that played 4 or more games in the season
QBstats = pd.DataFrame()
for name in statsDF.Name.unique():
    if len(statsDF.loc[statsDF.Name == name]) >= 4:
        if len(QBstats) == 0:
            QBstats = pd.DataFrame(statsDF.loc[statsDF.Name == name])
        else:
            current = pd.DataFrame(statsDF.loc[statsDF.Name == name])
            QBstats = pd.concat([QBstats, current])

QBstats1 = QBstats.sort_values(['Name', 'Week']).reset_index()  
QBstats1.loc[QBstats1.Name == 'Mitch Trubisky'].sort_values(['Week'])

In [ ]:
# get averages of stats prior to week listed
QBWeeks = QBstats1[['Name', 'Week']]
QBmerge = pd.merge(QBWeeks, QBstats1, on='Name')
QBedit = QBmerge[QBmerge['Week_x'] > QBmerge['Week_y']]
QBedit.rename(columns={'Week_x':'Week'}, inplace = True)
QBavg = QBedit.groupby(['Name','Week']).mean().reset_index()

# take out games 2 and 3 played by QBs (game 1 already removed)
filteredQBdata = pd.DataFrame({})
for name in QBavg.Name.unique():
    current = QBavg.loc[QBavg.Name == name]
    filteredQBdata = pd.concat([filteredQBdata, current.iloc[2:]])
filteredQBdata

In [ ]:
# present graph for MVP - show trend in one player who lowered yards/game and lowered fpts/game
brady = filteredQBdata.loc[filteredQBdata.Name == 'Tom Brady']
fig, (bradyYards, bradyPoints) = plt.subplots(nrows=1, ncols=2, figsize=(12,4))

bradyYards.scatter(x=brady['Week'], y=brady['PassYd'], marker='o', c='r', edgecolor='b')
bradyYards.set_title('Brady: Yards Per Week Average')
bradyYards.set_xlabel('Week')
bradyYards.set_ylabel('Average Yards Per Game Prior to Week')
bradyYards.set_xlim(xmin=0, xmax=18)
bradyYards.set_ylim(ymin=280, ymax=370)

bradyPoints.scatter(x=brady['Week'], y=brady['Fantasy Points'], marker='o', c='r', edgecolor='b')
bradyPoints.set_title('Brady: Fantasy Points Per Week Average')
bradyPoints.set_xlabel('Week')
bradyPoints.set_ylabel('Average Fantasy Points Per Game Prior to Week')
bradyPoints.set_xlim(xmin=0, xmax=18)
bradyPoints.set_ylim(ymin=15, ymax=30)

In [ ]:
# get avg QB points allowed/week by defense
defWeeks = defDF[['OPP', 'Week']]
defMerge = pd.merge(defWeeks, defDF, on='OPP')
filteredMerge = pd.DataFrame({})
defEdit = defMerge[defMerge['Week_x'] > defMerge['Week_y']]
defEdit.rename(columns = {'Week_x': 'Week'}, inplace=True)
for name in defEdit.Name.unique():
    if name not in filteredQBdata.Name.unique():
        defEdit = defEdit[defEdit.Name != name]
defEdit.drop_duplicates(subset=['OPP', 'Week', 'Week_y'], inplace=True)
defAvg = defEdit.groupby(['OPP', 'Week']).mean().reset_index()
defAvg = defAvg[['OPP', 'Week', 'QB Points Allowed']]


In [ ]:
# merge defense data with QB data
QBtoMerge = QBstats1[['Name', 'Week', 'OPP', 'Fantasy Points']]
df = pd.merge(filteredQBdata, QBtoMerge, on=['Name', 'Week'])
df.drop(['Week_y', 'index', 'Fantasy Points_x', 'RushTD'], axis=1, inplace=True)
df = pd.merge(df, defAvg, on=['OPP', 'Week'])
df.rename(columns={'Fantasy Points_y':'Fantasy Points'}, inplace = True)
df.loc[df.Name == 'Mitch Trubisky']

In [ ]:
with open("QBfantasyDF", 'wb') as picklefile:
    pkl.dump(df, picklefile)